# Notebook 16: Agentic Workflows - Multi-Tool MCP Agents

**Learning Objectives:**
- Build complex multi-tool agents
- Implement agent planning and reflection
- Chain tools for sophisticated workflows
- Compare agent architectures (ReAct, Plan-and-Execute)

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **small (CPU-friendly)** | llama3.2:3b | 2GB | 8GB | 8GB RAM, CPU | Better reasoning than 1b |
| **large (GPU-optimized)** | llama3.1:8b | 4.7GB | 16GB | 12GB VRAM (RTX 4080) | Multi-step planning |
| **SOTA (reference only)** | Claude 3.5 Sonnet | API | N/A | API key required | Best-in-class reasoning |

### Software Requirements
- Python 3.10+
- Completed Notebooks 14-15
- Libraries: `mcp`, `ollama`, `requests`

### Installation

```bash
pip install mcp ollama requests
ollama pull llama3.2:3b
```

## Agent Architectures

### 1. ReAct (Reason + Act)
- **Think** → **Act** → **Observe** → Repeat
- LLM reasons about what to do, takes action, observes result
- Most common pattern, used in Notebooks 14-15

### 2. Plan-and-Execute
- **Plan** all steps upfront → **Execute** sequentially
- Better for complex multi-step tasks
- Can replan if steps fail

### 3. Reflection
- **Act** → **Reflect** on results → **Improve**
- Self-critique and refinement
- Good for quality-critical tasks

### Comparison

| Architecture | Best For | Iterations | Token Cost |
|--------------|----------|------------|------------|
| **ReAct** | Simple tasks | 2-5 | Low |
| **Plan-and-Execute** | Complex workflows | 5-15 | Medium |
| **Reflection** | Quality tasks | 3-10 | High |

## Expected Behaviors

### Multi-Tool Workflows
- Agent uses 5-10 different tools in one task
- Tool calls ordered logically
- Results from one tool inform next tool choice

### Planning Phase
```
Planning...
Step 1: Search for information
Step 2: Calculate statistics
Step 3: Write report
Step 4: Save to file
```

### Reflection Phase
```
Reflection: Result looks incomplete
Improvement: Gather more data
Retry with better approach...
```

### Performance
- **llama3.2:3b**: 3-7 seconds per tool call
- **llama3.1:8b**: 2-4 seconds per tool call
- Complex tasks: 30-120 seconds total

### Common Observations
- Larger models plan more effectively
- Tool ordering improves with experience
- Reflection loops can get stuck (set max iterations)
- Clear task descriptions yield better plans

In [ ]:
import json
import random
import requests
from typing import List, Dict, Any
import ollama
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
random.seed(1103)

print("Multi-Tool Agent Tutorial - Setup Complete")

## Model Selection

In [ ]:
# CHOOSE YOUR MODEL:

# Option 1: small model (better reasoning than 1b)
MODEL_NAME = "llama3.2:3b"  # 2GB

# Option 2: large model (best for complex planning)
# MODEL_NAME = "llama3.1:8b"  # 4.7GB

print(f"Selected model: {MODEL_NAME}")

## Building a Comprehensive Tool Set

Let's create a rich set of tools for complex workflows.

In [ ]:
class ComprehensiveToolServer:
    """Server with tools for research, calculation, file ops, and web access."""
    
    def __init__(self, workspace: str = "./agent_workspace"):
        self.workspace = Path(workspace)
        self.workspace.mkdir(exist_ok=True)
        
        self.tools = [
            # Web & Search Tools
            {
                'type': 'function',
                'function': {
                    'name': 'web_search',
                    'description': 'Search the web for information (mock data)',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'query': {'type': 'string', 'description': 'Search query'}
                        },
                        'required': ['query']
                    }
                }
            },
            # Calculation Tools
            {
                'type': 'function',
                'function': {
                    'name': 'calculate',
                    'description': 'Evaluate mathematical expressions',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'expression': {'type': 'string', 'description': 'Math expression to evaluate'}
                        },
                        'required': ['expression']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'statistics',
                    'description': 'Calculate statistics (mean, median, std dev) for numbers',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'numbers': {'type': 'array', 'items': {'type': 'number'}}
                        },
                        'required': ['numbers']
                    }
                }
            },
            # File Operations
            {
                'type': 'function',
                'function': {
                    'name': 'save_report',
                    'description': 'Save a report to a file',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'filename': {'type': 'string'},
                            'content': {'type': 'string'}
                        },
                        'required': ['filename', 'content']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'read_report',
                    'description': 'Read a previously saved report',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'filename': {'type': 'string'}
                        },
                        'required': ['filename']
                    }
                }
            },
            # Data Processing
            {
                'type': 'function',
                'function': {
                    'name': 'summarize_text',
                    'description': 'Summarize long text into key points',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'text': {'type': 'string'},
                            'max_points': {'type': 'integer', 'description': 'Max bullet points (default 5)'}
                        },
                        'required': ['text']
                    }
                }
            },
            # Date/Time
            {
                'type': 'function',
                'function': {
                    'name': 'get_current_time',
                    'description': 'Get current date and time',
                    'parameters': {'type': 'object', 'properties': {}, 'required': []}
                }
            }
        ]
        
        self.function_map = {
            'web_search': self.web_search,
            'calculate': self.calculate,
            'statistics': self.statistics,
            'save_report': self.save_report,
            'read_report': self.read_report,
            'summarize_text': self.summarize_text,
            'get_current_time': self.get_current_time
        }
        
        print(f"ComprehensiveToolServer initialized")
        print(f"Workspace: {self.workspace.absolute()}")
        print(f"Available tools: {len(self.tools)}")
    
    def web_search(self, query: str) -> str:
        """Mock web search."""
        mock_results = {
            'python': 'Python is a high-level programming language. Created by Guido van Rossum in 1991.',
            'machine learning': 'Machine learning is a subset of AI. Popular frameworks include TensorFlow and PyTorch.',
            'climate change': 'Global temperatures have risen 1.1°C since pre-industrial times. Main cause is greenhouse gases.',
            'covid-19': 'COVID-19 is caused by SARS-CoV-2 virus. First detected in December 2019 in Wuhan, China.'
        }
        
        query_lower = query.lower()
        for key in mock_results:
            if key in query_lower:
                return f"Search results for '{query}': {mock_results[key]}"
        
        return f"Search results for '{query}': General information available. Topic is widely discussed."
    
    def calculate(self, expression: str) -> str:
        """Safely evaluate math expressions."""
        try:
            result = eval(expression, {"__builtins__": {}}, {})
            return f"{expression} = {result}"
        except Exception as e:
            return f"Error calculating '{expression}': {str(e)}"
    
    def statistics(self, numbers: List[float]) -> str:
        """Calculate statistics."""
        if not numbers:
            return "Error: Empty list"
        
        import statistics as stats
        result = {
            'count': len(numbers),
            'mean': round(stats.mean(numbers), 2),
            'median': stats.median(numbers),
            'stdev': round(stats.stdev(numbers), 2) if len(numbers) > 1 else 0,
            'min': min(numbers),
            'max': max(numbers)
        }
        return json.dumps(result, indent=2)
    
    def save_report(self, filename: str, content: str) -> str:
        """Save report to file."""
        try:
            path = self.workspace / filename
            with open(path, 'w', encoding='utf-8') as f:
                f.write(content)
            return f"Report saved to {filename} ({len(content)} characters)"
        except Exception as e:
            return f"Error saving report: {str(e)}"
    
    def read_report(self, filename: str) -> str:
        """Read report from file."""
        try:
            path = self.workspace / filename
            if not path.exists():
                return f"Error: Report '{filename}' not found"
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
            return content
        except Exception as e:
            return f"Error reading report: {str(e)}"
    
    def summarize_text(self, text: str, max_points: int = 5) -> str:
        """Create bullet point summary."""
        sentences = text.split('. ')
        key_points = sentences[:max_points]
        summary = '\n'.join(f"- {point.strip()}" for point in key_points if point.strip())
        return f"Summary ({max_points} points):\n{summary}"
    
    def get_current_time(self) -> str:
        """Get current date/time."""
        from datetime import datetime
        now = datetime.now()
        return f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"
    
    def execute_tool(self, tool_name: str, arguments: dict) -> str:
        """Execute a tool."""
        if tool_name not in self.function_map:
            return f"Error: Unknown tool '{tool_name}'"
        function = self.function_map[tool_name]
        return function(**arguments)

tool_server = ComprehensiveToolServer()

## Pattern 1: ReAct Agent (from Notebook 14)

In [ ]:
def react_agent(prompt: str, model: str = MODEL_NAME, max_iterations: int = 15) -> str:
    """
    ReAct: Reason → Act → Observe loop.
    """
    messages = [{'role': 'user', 'content': prompt}]
    
    print(f"\n{'='*70}")
    print(f"ReAct Agent")
    print(f"{'='*70}")
    print(f"Task: {prompt}\n")
    
    for iteration in range(1, max_iterations + 1):
        response = ollama.chat(
            model=model,
            messages=messages,
            tools=tool_server.tools
        )
        
        messages.append(response['message'])
        
        if not response['message'].get('tool_calls'):
            print(f"\nIteration {iteration}: Final Answer")
            final_answer = response['message']['content']
            print(f"Result: {final_answer}")
            print(f"\n{'='*70}\n")
            return final_answer
        
        print(f"Iteration {iteration}:")
        for tool_call in response['message']['tool_calls']:
            function_name = tool_call['function']['name']
            function_args = tool_call['function']['arguments']
            
            print(f"  Action: {function_name}({json.dumps(function_args, indent=4)})")
            
            result = tool_server.execute_tool(function_name, function_args)
            print(f"  Result: {result[:100]}..." if len(result) > 100 else f"  Result: {result}")
            
            messages.append({'role': 'tool', 'content': str(result)})
        print()
    
    return "Max iterations reached"

print("ReAct agent ready")

## Example: Research Report Workflow

In [ ]:
result = react_agent(
    """Create a research report about machine learning:
    1. Search for information about machine learning
    2. Summarize the findings
    3. Add the current date to the report
    4. Save it to 'ml_report.txt'
    """
)

## Pattern 2: Plan-and-Execute Agent

First plan all steps, then execute them.

In [ ]:
def plan_and_execute_agent(prompt: str, model: str = MODEL_NAME) -> str:
    """
    Plan-and-Execute: Create plan first, then execute steps.
    """
    print(f"\n{'='*70}")
    print(f"Plan-and-Execute Agent")
    print(f"{'='*70}")
    print(f"Task: {prompt}\n")
    
    # Phase 1: Planning
    print("Phase 1: Planning")
    print("-" * 70)
    
    planning_prompt = f"""Create a step-by-step plan to accomplish this task: {prompt}
    
Available tools:
- web_search: Search for information
- calculate: Do math
- statistics: Calculate stats
- save_report: Save file
- read_report: Read file
- summarize_text: Summarize text
- get_current_time: Get date/time

List the plan as numbered steps. Be specific about which tool to use."""
    
    plan_response = ollama.chat(
        model=model,
        messages=[{'role': 'user', 'content': planning_prompt}]
    )
    
    plan = plan_response['message']['content']
    print(f"Plan:\n{plan}\n")
    
    # Phase 2: Execution
    print("Phase 2: Execution")
    print("-" * 70)
    
    execution_prompt = f"""Execute this plan step by step:

{plan}

Original task: {prompt}

Use the available tools to complete each step."""
    
    messages = [{'role': 'user', 'content': execution_prompt}]
    
    for iteration in range(15):
        response = ollama.chat(
            model=model,
            messages=messages,
            tools=tool_server.tools
        )
        
        messages.append(response['message'])
        
        if not response['message'].get('tool_calls'):
            final_answer = response['message']['content']
            print(f"\nCompleted: {final_answer}")
            print(f"\n{'='*70}\n")
            return final_answer
        
        print(f"\nStep {iteration + 1}:")
        for tool_call in response['message']['tool_calls']:
            function_name = tool_call['function']['name']
            function_args = tool_call['function']['arguments']
            
            print(f"  Tool: {function_name}")
            result = tool_server.execute_tool(function_name, function_args)
            print(f"  Result: {result[:80]}..." if len(result) > 80 else f"  Result: {result}")
            
            messages.append({'role': 'tool', 'content': str(result)})
    
    return "Max iterations reached"

print("Plan-and-Execute agent ready")

In [ ]:
result = plan_and_execute_agent(
    """Research Python programming and create a summary report:
    1. Find information about Python
    2. Summarize into 3 key points
    3. Save to 'python_summary.txt'
    """
)

## Pattern 3: Reflection Agent

Agent reflects on results and improves.

In [ ]:
def reflection_agent(prompt: str, model: str = MODEL_NAME, max_reflections: int = 3) -> str:
    """
    Reflection: Act → Reflect → Improve loop.
    """
    print(f"\n{'='*70}")
    print(f"Reflection Agent")
    print(f"{'='*70}")
    print(f"Task: {prompt}\n")
    
    for reflection_round in range(max_reflections):
        print(f"\nRound {reflection_round + 1}:")
        print("-" * 70)
        
        # Execute task
        messages = [{'role': 'user', 'content': prompt}]
        result = None
        
        for iteration in range(10):
            response = ollama.chat(
                model=model,
                messages=messages,
                tools=tool_server.tools
            )
            
            messages.append(response['message'])
            
            if not response['message'].get('tool_calls'):
                result = response['message']['content']
                break
            
            for tool_call in response['message']['tool_calls']:
                function_name = tool_call['function']['name']
                function_args = tool_call['function']['arguments']
                
                tool_result = tool_server.execute_tool(function_name, function_args)
                messages.append({'role': 'tool', 'content': str(tool_result)})
        
        print(f"Result: {result}")
        
        # Reflect on result
        if reflection_round < max_reflections - 1:
            reflection_prompt = f"""Reflect on this result: {result}
            
Is it complete and high quality? What could be improved?
Answer with 'GOOD' if satisfactory, or suggest improvements."""
            
            reflection = ollama.chat(
                model=model,
                messages=[{'role': 'user', 'content': reflection_prompt}]
            )
            
            reflection_text = reflection['message']['content']
            print(f"\nReflection: {reflection_text}")
            
            if 'GOOD' in reflection_text.upper():
                print("\nSatisfactory result achieved!")
                break
            
            # Update prompt with reflection
            prompt = f"{prompt}\n\nPrevious attempt feedback: {reflection_text}\nPlease improve."
    
    print(f"\n{'='*70}\n")
    return result

print("Reflection agent ready")

In [ ]:
result = reflection_agent(
    "Create a detailed report about climate change with statistics and save it to 'climate_report.txt'",
    max_reflections=2
)

## Comparing Agent Patterns

In [ ]:
import time

test_task = "Search for COVID-19 information and create a 2-sentence summary. Save it to 'covid_summary.txt'."

print("\nComparing Agent Patterns")
print("="*70)

# Test ReAct
start = time.time()
react_result = react_agent(test_task)
react_time = time.time() - start

print(f"\n\nREACT Agent:")
print(f"  Time: {react_time:.2f}s")
print(f"  Result length: {len(react_result)} chars")

# Test Plan-and-Execute
start = time.time()
plan_result = plan_and_execute_agent(test_task)
plan_time = time.time() - start

print(f"\n\nPLAN-AND-EXECUTE Agent:")
print(f"  Time: {plan_time:.2f}s")
print(f"  Result length: {len(plan_result)} chars")

print("\n" + "="*70)
print("\nSummary:")
print(f"  ReAct: {react_time:.2f}s")
print(f"  Plan-and-Execute: {plan_time:.2f}s")
print(f"  Winner: {'ReAct' if react_time < plan_time else 'Plan-and-Execute'}")

## Advanced: Tool Chaining

Build workflows where one tool's output feeds into another.

In [ ]:
complex_task = """Complete this data analysis workflow:
1. Calculate statistics for these test scores: [78, 85, 92, 88, 95, 91, 87]
2. Search for information about 'educational assessment'
3. Write a report combining the statistics and search results
4. Save the report as 'education_analysis.txt'
5. Read back the file to confirm it was saved correctly
"""

result = react_agent(complex_task)

## Real-World Example: Data Pipeline

In [ ]:
data_pipeline_task = """Build a data analysis pipeline:
1. Calculate statistics for sales data: [1200, 1450, 1100, 1600, 1850, 1400, 1550]
2. Calculate the total sum
3. Calculate the average
4. Create a summary report with:
   - Total sales
   - Average sale
   - Highest sale
   - Lowest sale
   - Current date
5. Save to 'sales_report.txt'
"""

result = plan_and_execute_agent(data_pipeline_task)

## Agent Performance Monitoring

In [ ]:
# Check created reports
print("\nCreated Reports:")
print("="*70)

workspace = Path("./agent_workspace")
if workspace.exists():
    reports = list(workspace.glob("*.txt"))
    print(f"Total reports: {len(reports)}\n")
    
    for report in reports:
        size = report.stat().st_size
        print(f"  - {report.name} ({size} bytes)")
        
        # Show first 100 chars
        with open(report, 'r') as f:
            preview = f.read(100)
        print(f"    Preview: {preview}...\n")
else:
    print("No reports created yet")

## Exercises

1. **Custom Workflow**: Design a 5-step workflow using multiple tools
2. **Error Recovery**: Test how agents handle tool failures
3. **Optimization**: Which pattern is fastest for your use case?
4. **Tool Addition**: Add a new tool (e.g., `translate_text`) and test it
5. **Model Comparison**: Compare llama3.2:3b vs llama3.1:8b performance

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **Three agent patterns**: ReAct, Plan-and-Execute, Reflection

✅ **ReAct** is fastest for simple tasks

✅ **Plan-and-Execute** better for complex workflows

✅ **Reflection** improves quality through self-critique

✅ **Tool chaining** enables sophisticated workflows

✅ **Local models** (llama3) can handle multi-tool tasks

## Congratulations!

You've completed the Agentic Workflows series! You now know:

- MCP protocol fundamentals
- Building reusable MCP servers
- Three major agent architectures
- Complex multi-tool workflows
- Using local LLMs for agentic tasks

## Next Steps

- Build your own MCP server for a domain you care about
- Explore [MCP community servers](https://github.com/modelcontextprotocol/servers)
- Try commercial agents (Claude, GPT-4) for comparison
- Combine with Notebooks 10-13 for production workflows

## Resources

- [Model Context Protocol](https://modelcontextprotocol.io/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [Ollama Function Calling](https://ollama.com/blog/tool-support)
- [MCP Examples](https://github.com/modelcontextprotocol/examples)